In [8]:
import math

def fill_missing(trainingData, saveName):
        print("working on {}".format(trainingData))
        #traindata    
        with open(trainingData, 'r') as dataIn:
            data=dataIn.readlines()
        dataIn.close()

       
        #number of rows
        print("Rows Data {}".format(len(data)))
        
        cleanData_json={}
        amountOfFeatures=0
        highest=1.00000000000000e+99

        #train Data
        for i in range(0, len(data)):
            temp_data=data[i].replace("e+00","").replace("\n","").split("\t")
            if len(temp_data)==1:
                temp=temp_data[0]
                temp_data=temp.split()
#             print((temp_data))
            amountOfFeatures=len(temp_data)
            #hold indexes of empty values
            highest=1.00000000000000e+99
            count=0
            #find column that the empty value is in
            index_of_empty=[]
            index_with_values=[]
            for j in range(0, len(temp_data)):
#                 currVal=temp_data[j]
#                 temp_data[j]=currVal.replace("e+00","")
#train data
                temp_data[j].replace(" ","")
                if float(temp_data[j]) == float(highest):
                    index_of_empty.append(j)
                    count+=1
                else:
                    index_with_values.append(j)

            cleanData_json.update({i:[temp_data, count, index_of_empty, index_with_values]})

        print("Amount of features from Train{}".format(amountOfFeatures))


       #[ key for row in data : [original data , amount of empty data,[indexes of all empty values], [index with values]]  ]
        #now I have the 'clean' data but still with empty values...supposed to be set to the 1E9th


        def eucDist(primaryRow, index_withValues, iterationRow, index_NoValues):
            def dist(a,b):
                temp=float(a-b)
                return temp*temp

            sumDist =float(0)
            columnsUsed =[]
            if len(cleanData_json[primaryRow][0])!= len(cleanData_json[iterationRow][0]):
                return False
            else:
                #for each column that has values in rowA
                for index in index_withValues:

                    #if the current column is NOT null in the rowB
                    #that means this column for Row A is not null AND the this colmn is not null for Row B
                    #get euc dist
                    if index not in index_NoValues:
                        columnsUsed.append(index)
                        value_fromRowA=cleanData_json[primaryRow][0][index]
                        value_fromRowB=cleanData_json[iterationRow][0][index]
                        distance=dist(float(value_fromRowA),float(value_fromRowB) )
                        sumDist+=float(distance)
                EucDist=float(math.sqrt(sumDist))
                return [EucDist, columnsUsed]


        ##Finding KNN values for train data
        rows=list(cleanData_json.keys())
        Knn_NullValue_columns_json={}
        print("finding KNN Values FOR training Data")
        for cur_data_row in rows:
            #list of columns with values
            values=cleanData_json[cur_data_row][3]
            data_perRow={}
            #need to find euclidean distance from cur_data_row to every other row

            for knn_row in rows:
                if knn_row != cur_data_row:
                    temp_indices_of_nullData=cleanData_json[knn_row][2]


                    #returns [sumDist, columnsUsed]
                    response=eucDist(cur_data_row, values, knn_row, temp_indices_of_nullData)
                    #update the relatinship between current row and row we are comparing it to
                    data_perRow.update({knn_row: response})
            #update the data per row from every other row    
            Knn_NullValue_columns_json.update({cur_data_row: data_perRow})


        #i have all knn values of each row with what columns the value for derived each row FOR TRAIN
        # Knn_NullValue_columns_json

        print("filling empty values FOR training Data")
        #filling IN EMPTY VALUES FOR TRAINING DATA
        s=list(Knn_NullValue_columns_json.keys())
        print("number of rows to Iterate {}".format(len(s)))
        ValuesKnned=0
        for x in range (0, len(s)):
            data_for_currentRow=Knn_NullValue_columns_json[s[x]]
            indexofNullValues=cleanData_json[s[x]][2]

            #rows that contributed
            row_Contribution=[]
        #     print(indexofNullValues)

            for emptyValuekey in indexofNullValues:
                newValue=float(0)
                KNNdata_for_currentRow=Knn_NullValue_columns_json[s[x]]
        #knn dict holds k:(k-1)
        #so keys should not be hhave k 0:123  , 1:023 , 2:013
        #         print("type of empty Value {}".format(str(type(emptyValuekey))))
                for keys in KNNdata_for_currentRow:
                    ##cleanData_json[keys][3] in a list of columns that have a value that is not 'empty'
                    if emptyValuekey in cleanData_json[keys][3]:
                        valueFromOriginal=cleanData_json[keys][0][emptyValuekey]


                        temp=row_Contribution
                        row_Contribution.append(float(valueFromOriginal))


                row_Contribution.sort()
        #         print(row_Contribution)
                knn=math.sqrt(len(row_Contribution))
                temp=float(0)

                if knn !=0:
                    ValuesKnned+=1
                    for y in range(0, int(knn)):

                        temp+=float((row_Contribution[y]))

                    newValue=float(temp/float(knn))
    #                 print("changed value :{}".format(newValue))

                    cleanData_json[s[x]][0][emptyValuekey] =newValue   
        #             #at this stage I have the rows that can contribute to the empty value
                else:
                    print("a value was not found")
            if x%50==0:
                print("Done with row {}".format(str(x)))

        
        with open(saveName, 'w') as o:
            for everything in cleanData_json:
                o.write(str(list (cleanData_json[everything][0])))
        o.close()
        print("returned [cleanData_json]")

        



In [9]:
fill_missing("./datasets/MissingData1.txt","./custom_KNN_results/MissingData1.txt")
fill_missing("./datasets/MissingData2.txt","./custom_KNN_results/MissingData2.txt")
fill_missing("./datasets/MissingData3.txt","./custom_KNN_results/MissingData3.txt")


working on ./datasets/MissingData1.txt
Rows Data 242
Amount of features from Train14
finding KNN Values FOR training Data
filling empty values FOR training Data
number of rows to Iterate 242
Done with row 0
Done with row 50
Done with row 100
Done with row 150
Done with row 200
returned [cleanData_json]
working on ./datasets/MissingData2.txt
Rows Data 758
Amount of features from Train50
finding KNN Values FOR training Data
filling empty values FOR training Data
number of rows to Iterate 758
Done with row 0
Done with row 50
Done with row 100
Done with row 150
Done with row 200
Done with row 250
Done with row 300
Done with row 350
Done with row 400
Done with row 450
Done with row 500
Done with row 550
Done with row 600
Done with row 650
Done with row 700
Done with row 750
returned [cleanData_json]
working on ./datasets/MissingData3.txt
Rows Data 273
Amount of features from Train79
finding KNN Values FOR training Data
filling empty values FOR training Data
number of rows to Iterate 273
Don